In [15]:
import os
import json
from web3 import Web3
from pathlib import Path
from dotenv import load_dotenv
# Import the helper function from the pinata.py file
from pinata import convert_data_to_json, pin_file_to_ipfs, pin_json_to_ipfs
from solcx import compile_standard, install_solc


In [21]:
with open("./contracts/auction.sol", "r") as file:
    auction_file = file.read()

print("Installing solc...")
            # adjust version!
install_solc('0.8.17')


Installing solc...


Version('0.8.17')

In [25]:
# Configure compile standards (adjust version!)
compiled_sol = compile_standard(
    {
        "language": "Solidity",
                   # fileName.sol
        "sources": {"auction.sol": {"content": auction_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        }
    },
            # update version
    solc_version="0.8.17"
)

# Dump the compiled code to see the structure of the code
with open("./contracts/compiled/auction_python.json", "w") as file:
    json.dump(compiled_sol, file)

In [28]:
# Get the bytecode and abi (from json format)
# ["contracts"]["fileName.sol"]["contractName"]["evm"]["bytecode"]["object"]

bytecode = compiled_sol["contracts"]["auction.sol"]["Auction"]["evm"]["bytecode"]["object"]

abi = compiled_sol["contracts"]["auction.sol"]["Auction"]["abi"]

In [29]:
load_dotenv()

# Define and connect a new Web3 provider
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

In [30]:
################################################################################
# Load_Contract Function
################################################################################

def load_contract():

    # Create the contract with the provider using the abi and bytecode.
    contract = w3.eth.contract(
        bytecode=bytecode,
        abi=abi
    )

    return contract


# Load the contract
contract = load_contract()

In [33]:
# Get latest transaction count and use it as the nonce 
# Ganache address
address = "0x0e34A93a265aF40F44bfb27b6d880888139c7FB2"
nonce = w3.eth.getTransactionCount(address)

In [34]:
# Submit the transaction that deploys contract using the chainId, gasPrice, from (which addr) and nonce

# set deployment parameters
duration_sec = 600
beneficiary = "0xF45e042B2DD3570ddBe784317364cb1f5E7cfebf"


transaction = contract.constructor().buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": address,
        "nonce": nonce
    }
)

TypeError: Incorrect argument count.  Expected '3'.  Got '0'